In [149]:
#Version 0.0 ---comment preprocessing

import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import ndimage as nd
import re

Number_of_entry = 5000
#input data and clean up
data = pd.read_csv('aggression_annotated_comments.tsv', sep = '\t', nrows = Number_of_entry)

comment_data = data.comment
comment_data = np.asarray(comment_data)

PUNCTUATION_NO_SPACE = re.compile("[.;:!*`_'?,\"()\[\]]")
PUNCTUATION_SPACE = re.compile("-")
NEWLINE = re.compile("newlinetoken")
#This is more or less the nltk stop list with negations removed
skip = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", 'im',"youre",
        "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 
        'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 
        'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 
        "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 
        'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'if', 'or', 'ive',
        'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'between', 
        'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 
        'out', 'on', 'off', 'over', 'under', 'further', 'then', 'once', 'here', 'there', 'theres','when', 'where',
        'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such',
        'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 
        'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y','u','ur']

for i in range(comment_data.size):
    comment_data[i] = PUNCTUATION_NO_SPACE.sub("",comment_data[i].lower())
    comment_data[i] = NEWLINE.sub("",comment_data[i])
    comment_data[i] = PUNCTUATION_SPACE.sub(" ",comment_data[i].lower())
    comment_data[i] = comment_data[i].split()
    comment_data[i] = [word for word in comment_data[i] if word not in skip]
    
print(comment_data[0])

#stop_list = nltk.corpus.stopwords.words("english")

['not', 'creative', 'dictionary', 'definitions', 'terms', 'insurance', 'ensurance', 'properly', 'applied', 'destruction', 'dont', 'understand', 'fine', 'legitimate', 'criticism', 'ill', 'write', 'three', 'man', 'cell', 'bounty', 'hunter', 'easy', 'understand', 'ensured', 'insured', 'different', 'differ', 'assuredthe', 'sentence', 'quote', 'absolutely', 'neutral', 'arent', 'familiar', 'underlying', 'theory', 'strike', 'back', 'eg', 'submarines', 'employed', 'nuclear', 'warfare', 'guiding', 'insurance', 'nor', 'likely', 'three', 'man', 'cell', 'structure', 'kept', 'ira', 'broken', 'british', 'thats', 'fault', 'fine', 'fix', 'explain', 'but', 'nothing', 'personal', 'creative', 'itim', 'tired', 'arguing', 'article', 'multi', 'party', 'turns', 'plenty', 'use', 'mutually', 'mutual', 'apply', 'standard', 'id', 'moving', 'mutual', 'assured', 'destruction', 'talk', 'not', 'appealing', 'reagan', 'voters', 'biases', 'effectiveness', 'dropping', 'lythere', 'double', 'standard', 'edits', 'comes', '

In [145]:
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')

for i in range(comment_data.size):
    lemmatizer = WordNetLemmatizer()
    comment_data[i] = ' '.join([lemmatizer.lemmatize(word) for word in comment_data[i]])
    
print(comment_data[6])

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/Kiara2.0/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


removed followingall name early polish ruler ficticious therefore index naming oda von haldensleben husband dagome record first time ruler polanen tribe therefore indicated first document later developing land named polandthis quite comment name fictitious deserves least backing


In [146]:
annotation = pd.read_csv('aggression_annotations.tsv', sep = '\t', nrows =Number_of_entry*10)
target_raw = annotation.aggression
target_pro = np.zeros(Number_of_entry)

ctr = 0
for i in range(target_pro.size):
    aggr = 0
    for j in range(10):
        aggr += target_raw[ctr]
        ctr += 1
    if (aggr/10 > 0.5):
        target_pro[i] = 1
    else:
        target_pro[i] = 0
    
print(target_pro[6])

print(data.rev_id[6])
print(annotation.rev_id[60])

0.0
103624
103624


In [147]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

#print(type(comment_data[0]))

comment_data = np.asarray(comment_data)

"""def accuracy_test(c=0.01, ngram=3, trials=100):
    total = 0
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3))
    tfidf_vectorizer.fit(comment_data)
    comment_vectorized = tfidf_vectorizer.transform(comment_data)

    #print(comment_vectorized.shape)
    for i in range(trials):
        data_train, data_test, target_train, target_test = train_test_split(
            comment_vectorized, target_pro, test_size = 0.1)

        model = LinearSVC(C=0.01)
        model.fit(data_train, target_train)
        total += accuracy_score(target_test, model.predict(data_test))
    accuracy = total / trials
    print("Accuracy: ", accuracy)

accuracy_test()
"""

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3))
tfidf_vectorizer.fit(comment_data)
comment_vectorized = tfidf_vectorizer.transform(comment_data)

data_train, data_test, target_train, target_test = train_test_split(
    comment_vectorized, target_pro, test_size = 0.1)

model = LinearSVC(C=0.01)
model.fit(data_train, target_train)
accuracy = accuracy_score(target_test, model.predict(data_test))
print(accuracy)

0.912


In [148]:
#Words most associated with aggression/friendliness

feature_to_coef = {
    word: coef for word, coef in zip(
        tfidf_vectorizer.get_feature_names(), model.coef_[0]
    )
}

for best_positive in sorted(feature_to_coef.items(), key=lambda x: x[1], reverse=True)[:5]:
    print ("Friendly: ", best_positive)
    
for best_negative in sorted(feature_to_coef.items(), key=lambda x: x[1])[:5]:
    print ("Aggressive: ", best_negative)

Friendly:  ('welcome', 0.044892251957242175)
Friendly:  ('check one', 0.03023126133549706)
Friendly:  ('fuck family', 0.027455303211724336)
Friendly:  ('page', 0.027265067783274997)
Friendly:  ('line', 0.02647619226522153)
Aggressive:  ('but', -0.02304388471958091)
Aggressive:  ('list', -0.01991580547643531)
Aggressive:  ('article', -0.019876091397173695)
Aggressive:  ('support', -0.01831803756604175)
Aggressive:  ('attack', -0.017870539244158067)
